# Bronze Layer - Raw Data Ingestion
## FinchMart Sales ETL Pipeline

This notebook ingests raw CSV sales transaction files using Spark Structured Streaming and stores them in Delta Lake format in the Bronze layer.

**Architecture Decision:** Using Structured Streaming with `readStream` to simulate real-time ingestion of CSV files, treating each file as a micro-batch representing 5 minutes of sales data.

In [ ]:
# Import required libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_timestamp, input_file_name
from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType, TimestampType
from delta import configure_spark_with_delta_pip
import os

In [ ]:
# Initialize Spark Session with Delta Lake support
builder = SparkSession.builder \
    .appName("FinchMart-Bronze-Layer") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.streaming.schemaInference", "true")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

print(f"Spark Version: {spark.version}")
print(f"Delta Lake configured successfully")

In [ ]:
# Define paths
BASE_PATH = "/home/ubuntu/dataengineer-transformations-python/finchmart_sales_etl"
RAW_DATA_PATH = f"{BASE_PATH}/data/raw"
BRONZE_PATH = f"{BASE_PATH}/data/bronze/sales_transactions"
BRONZE_CHECKPOINT = f"{BASE_PATH}/data/bronze/checkpoints/sales_transactions"

print(f"Raw Data Path: {RAW_DATA_PATH}")
print(f"Bronze Layer Path: {BRONZE_PATH}")

In [ ]:
# Define schema for sales transactions
# Explicit schema definition ensures data quality and type safety
sales_schema = StructType([
    StructField("transaction_id", StringType(), False),
    StructField("timestamp", StringType(), False),  # Will be converted to TimestampType in Silver
    StructField("customer_id", StringType(), False),
    StructField("product_id", StringType(), False),
    StructField("product_category", StringType(), True),
    StructField("price", FloatType(), True),
    StructField("quantity", IntegerType(), True),
    StructField("payment_method", StringType(), True),
    StructField("store_location", StringType(), True)
])

print("Schema defined for sales transactions")

In [ ]:
# Read streaming data from CSV files
# Using maxFilesPerTrigger=1 to simulate real-time processing of each 5-minute batch
raw_sales_stream = spark.readStream \
    .format("csv") \
    .option("header", "true") \
    .schema(sales_schema) \
    .option("maxFilesPerTrigger", 1) \
    .load(f"{RAW_DATA_PATH}/Mock_Sales_Data*.csv")

print("Streaming source configured")

In [ ]:
# Add metadata columns for data lineage and audit
bronze_sales_stream = raw_sales_stream \
    .withColumn("ingestion_timestamp", current_timestamp()) \
    .withColumn("source_file", input_file_name())

print("Metadata columns added for data lineage")

In [ ]:
# Write to Bronze layer as Delta table
# Using append mode to accumulate all raw data
# Checkpointing ensures exactly-once processing semantics
bronze_query = bronze_sales_stream.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", BRONZE_CHECKPOINT) \
    .option("path", BRONZE_PATH) \
    .trigger(processingTime="10 seconds") \
    .start()

print("Bronze layer streaming query started")
print(f"Query ID: {bronze_query.id}")
print(f"Status: {bronze_query.status}")

In [ ]:
# Monitor the streaming query
import time

# Wait for processing to complete (adjust timeout as needed)
timeout = 60  # seconds
start_time = time.time()

while bronze_query.isActive and (time.time() - start_time) < timeout:
    print(f"Query is active. Progress: {bronze_query.lastProgress}")
    time.sleep(5)

# Stop the query after processing
bronze_query.stop()
print("Bronze layer ingestion completed")

In [ ]:
# Verify Bronze layer data
bronze_df = spark.read.format("delta").load(BRONZE_PATH)

print(f"Total records in Bronze layer: {bronze_df.count()}")
print("\nSchema:")
bronze_df.printSchema()
print("\nSample data:")
bronze_df.show(5, truncate=False)

In [ ]:
# Data quality checks
print("=== Bronze Layer Data Quality Report ===")
print(f"Total transactions: {bronze_df.count()}")
print(f"Distinct transaction IDs: {bronze_df.select('transaction_id').distinct().count()}")
print(f"Date range: {bronze_df.selectExpr('min(timestamp)', 'max(timestamp)').first()}")
print(f"Source files processed: {bronze_df.select('source_file').distinct().count()}")

# Check for nulls in critical columns
from pyspark.sql.functions import col, sum as _sum, when

null_counts = bronze_df.select([
    _sum(when(col(c).isNull(), 1).otherwise(0)).alias(c)
    for c in bronze_df.columns
])

print("\nNull counts by column:")
null_counts.show()

## Summary

**Bronze Layer Ingestion Complete:**
- Raw CSV files ingested using Spark Structured Streaming
- Data stored in Delta Lake format for ACID transactions
- Metadata columns added for data lineage tracking
- Checkpoint mechanism ensures exactly-once processing
- Ready for Silver layer transformation and cleansing